In [1]:
# user interface demo
# core functionality:
#   midi file to sheet music
#   creating the structure for markov's chain
#   predicting next note based on last note
# convert back to midi file
# output and download

# unit tests
# test coverage
# code quality

Imports

In [1]:
from mido import MidiFile, MidiTrack, Message
import random


import user_interface

Converting a midi number to pitch

In [2]:
def note(midi_note):
    notes = [
        'C', 'C#', 'D', 'D#', 'E', 'F', 'F#', 'G', 'G#', 'A', 'A#', 'B'
    ]
    octave = (midi_note // 12) - 2  
    note_name = notes[midi_note % 12]
    return f"{note_name}{octave}"

Converting pitch to midi number

In [3]:
def midi(note):
    notes = [
        'C', 'C#', 'D', 'D#', 'E', 'F', 'F#', 'G', 'G#', 'A', 'A#', 'B'
    ]
    i=0
    while notes[i]!=note[:-1]:
        i+=1
        
    return (int(note[-1])+2) * 12 + i

Converting midi to pitch, duration

In [4]:
def midi_to_pitch(file):
    midi_file = MidiFile(file)
    pitches = []
    duration = []
    midi_pitches = []
    for track in midi_file.tracks:
        for msg in track:
            if msg.type == 'note_on':  # note_on event
                #pitch = midi_to_pitch(msg.note)  # Convert MIDI number to pitch name
                pitches.append(note(msg.note))
                midi_pitches.append(msg.note)
            if msg.type == 'note_off':  # note_on event
                duration.append(msg.time)
            # if msg.type == 'program_change':
            #     print(msg.program)
    return pitches, duration

Converting pitch, duration to midi

In [5]:
def pitch_to_midi(pitches, instrument):
    notes = []
    # Create a new MIDI file
    midi_file = MidiFile()
    track = MidiTrack()
    midi_file.tracks.append(track)
    track.append(Message('program_change', program=instrument))
    for i in range(len(pitches)):
        notes.append((midi(pitches[i]), 300))
        #notes.append((midi(pitches[i]), duration[i]))
    for pitch, d in notes:
        track.append(Message('note_on', note=pitch, velocity=64, time=0))  # Start the note
        track.append(Message('note_off', note=pitch, velocity=64, time=d))  # Stop the note after 'duration' ticks

    midi_file.save('output.mid')

In [6]:
class Node:
    def __init__(self, note):
        self.note = note
        self.probability = 0
        self.freq = 1
        self.children = {}
    
    def add_child(self, note):
        self.children[note] = Node(note)

    
    def new_notes(self, note2, note3):
        self.freq += 1
        if note2 in self.children:
            for child in self.children:
                if child == note2:
                    self.children[child].freq += 1
                    if note3 in self.children[child].children:
                        for child2 in self.children[child].children:
                            if child2 == note3:
                                self.children[child].children[child2].freq += 1
                    else:
                        self.children[child].add_child(note3)
        else:
            self.add_child(note2)
            self.children[note2].add_child(note3)

In [7]:
def get_prob():
    for root in roots.values():
        for node2 in root.children:
            root.children[node2].probability = root.children[node2].freq/root.freq
            for node3 in root.children[node2].children:
                root.children[node2].children[node3].probability = root.children[node2].children[node3].freq/root.children[node2].freq


In [8]:
def print_trie():
    for root in roots.values():
        print(root.note)
        for node2 in root.children:
            print("\t", node2, round(root.children[node2].probability, 2))
            for node3 in root.children[node2].children:
                print("\t\t", node3, round(root.children[node2].children[node3].probability, 2))

        print("\n \n")

Creating the probability trie

In [13]:
#pitches = ["C", "A", "D", "C", "B", "A", "C", "A", "E", "C", "B", "E", "C", "B", "A"]
roots = {}
def trie(pitches, duration):
    for i in range(len(pitches)-2):
        if pitches[i] not in roots:
            roots[pitches[i]] = Node(pitches[i])
            roots[pitches[i]].freq -= 1
        roots[pitches[i]].new_notes(pitches[i+1], pitches[i+2])
    get_prob()

In [ ]:
pitches, duration = midi_to_pitch('vivaldi_spring.mid')
trie(pitches, duration)
pitches, duration = midi_to_pitch('vivaldi_summer.mid')
trie(pitches, duration)
pitches, duration = midi_to_pitch('vivaldi_winter.mid')
trie(pitches, duration)
pitches, duration = midi_to_pitch('vivaldi_autumn.mid')
trie(pitches, duration)
pitches, duration = midi_to_pitch('sonata13.mid')
trie(pitches, duration)

In [20]:
print_trie()

C#3
	 E3 0.15
		 G#3 0.63
		 A3 0.13
		 G3 0.03
		 B2 0.11
		 F3 0.05
		 D3 0.03
		 A2 0.03
	 F#3 0.05
		 A3 0.5
		 F#3 0.5
	 G3 0.02
		 A#3 0.5
		 D#1 0.17
		 D#3 0.17
		 G#3 0.17
	 D#3 0.15
		 C#3 0.62
		 C#4 0.05
		 D#3 0.1
		 G2 0.05
		 A#2 0.08
		 A2 0.03
		 G#2 0.03
		 F2 0.03
		 D#2 0.03
	 C#3 0.33
		 C#3 0.6
		 G#2 0.05
		 F#3 0.07
		 D#3 0.12
		 A2 0.05
		 G#3 0.02
		 D3 0.05
		 C4 0.01
		 F3 0.01
		 C#4 0.01
		 D#2 0.01
	 G#2 0.02
		 G#2 1.0
	 A2 0.02
		 A2 0.5
		 G2 0.5
	 G#3 0.01
		 A3 0.33
		 G3 0.33
		 C#3 0.33
	 G1 0.01
		 D2 0.5
		 C3 0.5
	 D3 0.02
		 D3 0.5
		 G2 0.5
	 B2 0.01
		 D3 1.0
	 A#2 0.02
		 D#2 0.17
		 D#3 0.17
		 C#2 0.17
		 G#1 0.17
		 C3 0.33
	 C3 0.02
		 C2 0.4
		 C#2 0.4
		 D#4 0.2
	 D#2 0.05
		 G#2 0.14
		 C#3 0.14
		 A#1 0.07
		 G#1 0.07
		 A#2 0.14
		 C3 0.07
		 G3 0.14
		 A#3 0.14
		 G#3 0.07
	 F2 0.01
		 A#0 0.5
		 F3 0.5
	 C#2 0.02
		 C#2 0.25
		 F2 0.25
		 D#1 0.25
		 A#2 0.25
	 F3 0.02
		 F2 0.25
		 C#3 0.75
	 A#1 0.02
		 C4 0.25
		 C#3 0.5
		 D#

In [21]:
input = ["C#3", "E3"]
melody = input.copy()

In [22]:
for i in range(50):
    note1 = melody[i]
    note2 = melody[i+1]
    node2 = roots[note1].children[note2]
    dist = list(node2.children.keys())
    weights = list([j.probability*100 for j in node2.children.values()])
    pred_note = random.choices(dist, weights = weights, k = 1)[0]
    melody.append(pred_note)
    #print(pred_note)
print(melody)

['C#3', 'E3', 'G#3', 'C#4', 'D#3', 'F#3', 'C4', 'D#4', 'E4', 'D#4', 'E4', 'D#4', 'C#4', 'D#4', 'C#4', 'D#4', 'C#4', 'D#4', 'C#4', 'D#4', 'C#4', 'C4', 'D#4', 'D#4', 'F4', 'D#4', 'D4', 'D#4', 'D4', 'D#4', 'F4', 'G4', 'A4', 'B4', 'F4', 'E4', 'D4', 'E4', 'D4', 'C#4', 'D4', 'E4', 'F4', 'G4', 'F4', 'D#4', 'D4', 'D#4', 'D#4', 'A#4', 'G#4', 'F#4']


In [23]:
instrument = 26
pitch_to_midi(melody, instrument)